# Assignment 2: Graph Search
In this assignment you will be implementing a variety of graph search algorithms, with the eventual goal of solving tridirectional search.

Before you start, you will need to install:

1. [networkx](http://networkx.github.io/), which is a package for processing networks. This assignment will be easier if you take some time to test out and get familiar with the [basic methods](https://networkx.github.io/examples.html) of networkx.

2. [matplotlib](http://matplotlib.org/downloads.html) for basic network visualization.

We will be using two undirected networks for this assignment: a simplified map of Romania (from Russell and Norvig) and a full street map of Atlanta.

In [28]:
from __future__ import division
import random
import matplotlib.pyplot as plt
import pickle
import sys
sys.path.append('lib')
import networkx

In [29]:
"""Romania map data from Russell and Norvig, Chapter 3."""
romania = pickle.load(open('romania_graph.pickle', 'rb'))

Warmups
------
We'll start by implementing some simpler search and optimization methods before the real exercises.

Warmup 1: Priority queue
----------------------
5 points

In all searches that involve calculating path cost or heuristic (e.g. uniform-cost), we have to order our search frontier. It turns out the way that we do this can impact our overall search runtime. 

To show this, you'll implement a [priority queue](https://en.wikipedia.org/wiki/Priority_queue) and demonstrate its performance benefits. For large graphs, sorting all input to a priority queue is impractical. As such, the datastructure you implement should have an amortized O(1) insertion and removal time. It should do better than the queue you've been provided in InsertionSortQueue().

Hints: 
1. The [heapq](https://docs.python.org/2/library/heapq.html) module has been imported for you.
2. Each edge has an associated weight.

In [30]:
import heapq

class PriorityQueue():
    """Implementation of a priority queue
    to store nodes during search."""
    # TODO: finish this class

    # HINT look up the module heapq.

    def __init__(self):
        self.queue = []
        self.current = 0
        self.path = []
        self.cost = 0

    def next(self):
        if self.current >=len(self.queue):
            self.current
            raise StopIteration

        out = self.queue[self.current]
        self.current += 1

        return out

    def pop(self):
        # TODO: finish this
        self.current -= 1
        return heapq.heappop(self.queue)

    def get(self):
        return heapq.heappop(self.queue)[1]

    def __iter__(self):
        return self

    def __str__(self):
        return 'PQ:[%s]'%(', '.join([str(i) for i in self.queue]))

    def append(self, node):
        # TODO: finish this
        heapq.heappush(self.queue,(node))
        self.current += 1

    def push(self, cost,node,path):
        # TODO: finish this
        self.cost = cost
        self.path = path
        heapq.heappush(self.queue,(self.cost,node,self.path))
        self.current += 1

    def put(self, item, priority):
        heapq.heappush(self.queue, (priority, item))

    def empty(self):
        return len(self.queue) == 0

    def __contains__(self, key):
        #self.current = 0
        return key in [n for v,n in self.queue]

    def astarContains(self, key):
        #self.current = 0
        return key in [y for x,y,z in self.queue]

    def __eq__(self, other):
        return self == other

    def clear(self):
        self.queue = []

    def has_element(self):
        if len(self.queue) > 0:
            return True
        return False

    __next__ = next

In [31]:
from search_tests import priority_queue_tests

priority_queue_tests(PriorityQueue)

PriorityQueue tests passed


##### Warm-up 2: BFS
----------
5 pts

To get you started with handling graphs in networkx, implement and test breadth-first search over the test network. You'll do this by writing two methods: 
1. breadth_first_search_goal, which returns the goal node and the set of all nodes explored, but no path.
2. breadth_first_search, which returns the path and the set of all nodes explored.

Hint: networkx.edges() will return all edges connected to a given node.

In [32]:
 pathes = pickle.load(open('romania_test_paths.pickle', 'r'))

In [33]:
pathes[('a', 'm')]


(['a', 't', 'l', 'm'], ['a', 's', 'z', 't', 'r', 'o', 'f', 'l'])

In [34]:
#pq.length

In [35]:
def breadth_first_search_goal(graph, start, goal):
    """Run a breadth-first search from start
    to goal and return the goal as well as
    all nodes explored."""
    if start == goal: return None
    frontier = [start]
    explored = []

    while len(frontier) != 0:
        cur_node = frontier.pop(0)
        explored.append(cur_node)
        neighbours = graph.neighbors(cur_node)
        for neighbour in neighbours:
            if (neighbour not in explored) and (neighbour not in frontier):
                frontier.append(neighbour)
                if (neighbour == goal):
                    neighbour = goal
                    frontier[:] = []
    
    return goal, explored

In [36]:
def breadth_first_search(graph, start, goal):
    """Run a breadth-first search from start
    to goal and return the path as well as 
    all nodes explored."""
    if start == goal: return [],[]
    frontier = [start]
    explored = []
    path = []
    while len(frontier) != 0:
        cur_node = frontier.pop(0)
        explored.append(cur_node)
        neighbours = graph.neighbors(cur_node)
        for neighbour in neighbours:
            if (neighbour not in explored) and (neighbour not in frontier):
                if(neighbour == goal):
                    return pathFinder(graph, start, goal),explored
                else:
                    frontier.append(neighbour)



        
   

In [37]:
def pathFinder(graph, start, goal):
     # Finding the path
    path_queue = [start]
    path = []
    while len(path_queue) != 0:
            path = path_queue.pop(0)
            cur_node = path[-1]
            if (cur_node == goal): return path
            neighbours = graph.neighbors(cur_node)
            for neighbour in neighbours:
                updated_path = list(path)
                updated_path.append(neighbour)
                #print updated_path
                path_queue.append(updated_path)
                

In [38]:
# This function exists to help you visually debug your code.
# Feel free to modify it in any way you like.
# graph should be a networkx graph
# node_positions should be a dictionary mapping nodes to x,y coordinates
def draw_graph(graph, node_positions={}, start=None, goal=None, explored=[], path=[]):

    if not node_positions:
        node_positions = networkx.spring_layout(graph)

    networkx.draw_networkx_nodes(graph, node_positions)
    networkx.draw_networkx_edges(graph, node_positions, style='dashed')
    
    networkx.draw_networkx_nodes(graph, node_positions, nodelist=explored, node_color='g') 

    if path:
        edges = [(path[i], path[i+1]) for i in range(0, len(path)-1)]
        networkx.draw_networkx_edges(graph, node_positions, edgelist=edges, edge_color='b')
   
    if start:
        networkx.draw_networkx_nodes(graph, node_positions, nodelist=[start], node_color='b')
    
    if goal:
        networkx.draw_networkx_nodes(graph, node_positions, nodelist=[goal], node_color='y')
    
    labels={}
    for node in romania.nodes():
        labels[node] = node
    networkx.draw_networkx_labels(graph,node_positions,labels,font_size=16)
    plt.plot()
    plt.show()

In [39]:
path, explored = breadth_first_search(romania, 'h', 's') 
print path
print explored
pathes[('h', 's')]

['h', 'u', 'b', 'f', 's']
['h', 'u', 'e', 'b', 'v', 'p', 'g', 'f']


(['h', 'u', 'b', 'f', 's'], ['h', 'u', 'e', 'b', 'v', 'p', 'g', 'f'])

In [40]:
"""Testing and visualizing breadth-first search
in the notebook."""
start = 'h'
goal = 's'
#%debug
#%debug --breakpoint search_tests.py:58
#%run -d
goal, explored = breadth_first_search_goal(romania, start, goal) 
path, explored = breadth_first_search(romania, start, goal) 
node_locations = {n: romania.node[n]['position'] for n in romania.node.keys()}
draw_graph(romania, node_locations, start, goal, explored, path)

In [41]:
print explored
print path

['h', 'u', 'e', 'b', 'v', 'p', 'g', 'f']
['h', 'u', 'b', 'f', 's']


In [42]:
from search_tests import bfs_tests

bfs_tests(breadth_first_search)

BFS tests passed.


Warmup 3: Uniform-cost search
----------------------------
10 points

Implement uniform-cost search using PriorityQueue() as your frontier. From now on, PriorityQueue() should be your default frontier. 

uniform_cost_search() should return the same arguments as breadth-first search: the path to the goal node, the set of all nodes explored, and the total cost of the path.

In [43]:
def uniform_cost_search(graph, start, goal):
    """Run uniform-cost search from start
    to goal and return the path, the nodes
    explored, and the total cost."""
    # TODO: finish this function
    if start == goal: return [],[],0
    frontier = PriorityQueue()
    frontier.push(0,start,[]) #frontier.append(cost,node,path)
    explored = {}
 
    while frontier.has_element():
        
        cost,node,path = frontier.pop()
        
        if (len(explored) > 0):
            if (explored.has_key(node)) and (explored[node]<cost):
                continue

        path = path + [node]
        
   
        neighbours = graph.edges(node)
        for child in neighbours:
            child_node = child[1]
            child_cost = graph.edge[node][child_node]['weight']
            if child_node not in explored: 
                frontier.push(cost + child_cost,child_node,path)
                if child_node == goal:
                    #if path[0] == 'z' and path[-1] == 'h':
                        #print path
                    return path,list(explored.keys()),cost
        explored[node] = cost 



In [44]:
from search_tests import ucs_tests
ucs_tests(uniform_cost_search)
#pathes[('z', 'h')][0]

Uniform cost search tests passed


Warmup 4: A\* search
------------------
10 points

Implement A\* search using Euclidean distance as your heuristic. You'll need to implement heuristic_euclid() then pass that function to a_star() as the heuristic parameter. We provide null_heuristic() as a baseline heuristic to test against when calling a_star_tests().

In [45]:
romania.edge['a']['s']['weight']
romania.node['a']['position']

(91, 492)

In [46]:
def null_heuristic(graph, u, v, goal):
    """Return 0 for all nodes."""
    return 0

In [47]:
def heuristic_euclid(graph, u, v, goal):
    """Return the Euclidean distance from
    the node to the goal. u is current node,
    v is node under consideration."""
    import math
    return math.sqrt( (graph.node[goal]['position'][0] - graph.node[v]['position'][0])**2 +  (graph.node[goal]['position'][1] - graph.node[v]['position'][1])**2  )


In [48]:
def a_star(graph, start, goal, heuristic):
    """Run A* search from the start to
    goal using the specified heuristic
    function, and return the final path
    and the nodes explored."""
    
    def reconstruct_path(came_from, start, goal):
        if start == goal:
            return []
        current = goal
        path = [current]
        while current != start:
            current = came_from[current]
            path.append(current)
        path.reverse()
        return path

    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    explored = [start]

    while not frontier.empty():
        current = frontier.get()

        if current not in explored:
            explored.append(current)

        if current == goal:
            break

        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.edge[current][next]['weight']
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(graph, current, next, goal)
                frontier.put(next, priority)
                came_from[next] = current

    return reconstruct_path(came_from, start, goal), explored

In [49]:
from search_tests import a_star_tests

#%debug
a_star_tests(a_star, null_heuristic, heuristic_euclid)
pathes[('z', 'h')]

A star null_heuristic search tests passed
A star euclidean_dist_heuristic search tests passed


(['z', 'a', 's', 'f', 'b', 'u', 'h'],
 ['z', 'a', 'o', 's', 't', 'r', 'f', 'l', 'p', 'c', 'b', 'm', 'd', 'u'])

Exercises
-------

The following exercises will require you to implement several kinds of bidirectional and tridirectional searches.

For the following exercises, we will be using Atlanta [OpenStreetMap](wiki.openstreetmap.org/) data for our searches. If you want to run tests in iPython notebook using this data (rather than just calling the tests in search_tests), you'll need to load the data from file in the cell below.

In [50]:
"""Loading Atlanta map data."""
atlanta = pickle.load(open('atlanta_osm.pickle','r'))

Visualizing search results
---

When using a geographic network, you may want to visualize your searches. We can do this by converting the search results to a [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) file which we then visualize on [Gist](https://gist.github.com/) by [importing](https://github.com/blog/1576-gist-meets-geojson) the file.

We provide a method for doing this in visualize_graph.py called plot_search(), which takes as parameters the graph, the name of the file to write, the nodes on the path, and the set of all nodes explored. This produces a GeoJSON file named as specified, which you can upload to Gist to visualize the search path and explored nodes.

In [51]:
"""Example of how to visualize search results
with two sample nodes in Atlanta."""
from visualize_graph import plot_search
path, explored = bidirectional_ucs(atlanta, '69244359', '557989279')
plot_search(graph, 'atlanta_search.json', path, explored)
# then upload 'atlanta_search.json' to Gist

NameError: name 'bidirectional_ucs' is not defined

Exercise 1: Bidirectional uniform-cost search
-------
15 points

Implement bidirectional uniform-cost search. Remember that this requires starting your search at both the start and end states.

This function will return the goal, the set of explored nodes from the start node's search, and the set of explored nodes from the goal node's search.

In [54]:
def bidirectional_ucs(graph, start, goal):
    """Run bidirectional uniform-cost search
    between start and goal, and return the path,
    the nodes explored from start-initial
    search, and the nodes explored from the
    goal-initial search."""
    import copy
    from itertools import groupby

    def remove_dups(lst):
        return [k for k,items in groupby(lst)]

    def getCost(graph, path):
        cost = 0
        prevNode = "yolo"
        for i in path:
            if prevNode == "yolo":
                prevNode = i
            else:
                cost += graph.edge[prevNode][i]['weight']
                prevNode = i
        return cost

    def getPath(child, goal, goal_paths):
        if child == goal:
            return []
        path = [child]
        while path[-1] != goal:
            path.append(goal_paths[path[-1]])
        return path


    def build_next_level(paths, frontier, explored, graph, goal):
        p = copy.deepcopy(paths)
        ex = copy.deepcopy(explored)
        mf = copy.deepcopy(frontier)
        cost, node, path = mf.pop()

        for child_node in graph.neighbors(node):
            #child_node = child[1]
            child_cost = graph.edge[node][child_node]['weight']

            if (child_node not in ex and (child_node not in p or ex[p[child_node]] + graph.edge[p[child_node]][child_node]['weight'] > cost + child_cost)) or (child_node in ex and ex[child_node] > cost + child_cost):
                mf.push(cost + child_cost,child_node,path + [child_node])
                p[child_node] = node

        ex[node] = cost
        return p, mf, ex

    if start == goal:
            return [],([],[])

    #if they are neighbors
    if graph.neighbors(start).__contains__(goal):
        return [start,goal], ([start,goal],[goal,start])

    middleNodes = []
    start_paths = { start:None } # node:parent
    goal_paths = { goal:None }

    startFrontier = PriorityQueue()
    startFrontier.push(0,start,[]) #cost, node, path
    goalFrontier = PriorityQueue()
    goalFrontier.push(0,goal,[]) #cost, node, path
    #start_last_level = [start]
    #goal_last_level = [goal]

    sExplored = {}
    gExplored = {}

    while startFrontier.has_element() and goalFrontier.has_element():
        # get shared nodes
        startNodes = set(sExplored.keys())
        goalNodes = set(gExplored.keys())
        middleNodes = startNodes.intersection(goalNodes)
        if middleNodes: break


        # build next level/depth decide on which to expand
        sCost, sNode, sPath = startFrontier.pop()
        startFrontier.push(sCost, sNode, sPath)
        gCost, gNode, gPath = goalFrontier.pop()
        goalFrontier.push(gCost, gNode, gPath)

        if sCost <= gCost: #expand from start side
            start_paths, startFrontier, sExplored = build_next_level(start_paths, startFrontier, sExplored, graph, goal)
        else:
            goal_paths, goalFrontier, gExplored = build_next_level(goal_paths, goalFrontier, gExplored, graph, goal)
    else: # exited without break, empty path
        return [],([],[])

    # build the result

    middle = list(middleNodes)[0]
    #from middle to a
    res = []
    c = middle
    while c != start:
        c = start_paths[c]
        res.insert(0,c)

    res.append(middle)

    #from middle to b
    c = middle
    while c != goal:
        c = goal_paths[c]
        res.append(c)

    #look for shortcut
    #res is original path
    while startFrontier.has_element():
        cost, node, path = startFrontier.pop()
        if gExplored.__contains__(node):
            tempPath = [start]
            tempPath.extend(path)
            tempPath.extend(getPath(node, goal,goal_paths))
            tempPath = remove_dups(tempPath)
            tempCost = cost + gExplored[node]
            oCost = getCost(graph, remove_dups(res))
            if tempCost < oCost:
                res = tempPath

    return res, (sExplored,gExplored)
    #return path, (start_explored, goal_explored)

Works in IDE for some reason

In [55]:
from search_tests import bidirectional_tests

bidirectional_tests(bidirectional_ucs)

TypeError: 'NoneType' object has no attribute '__getitem__'

Exercise 2: Bidirectional A\* search
-------
20 points

Implement bidirectional A\* search. Remember that you need to calculate a heuristic for both the start-to-goal search and the goal-to-start search.

This function will return the final search path, the set of nodes explored during the start-to-goal search, and the set of nodes explored during the goal-to-start search.

In [ ]:
def bidirectional_a_star(graph, start, goal):
    """Run bidirectional uniform-cost search
    between start and goal, and return the path,
    the nodes explored from start-initial
    search, and the nodes explored from the
    goal-initial search."""

    from itertools import groupby

    def remove_dups(lst):
        return [k for k,items in groupby(lst)]

    def getCost(graph, path):
        cost = 0
        prevNode = "yolo"
        for i in path:
            if prevNode == "yolo":
                prevNode = i
            else:
                cost += graph.edge[prevNode][i]['weight']
                prevNode = i
        return cost

    def getPath(child, goal, goal_paths):
        if child == goal:
            return []
        path = [child]
        while path[-1] != goal:
            path.append(goal_paths[path[-1]])
        return path

    def heuristic(graph, u, v, goal):
        return math.sqrt( (graph.node[goal]['position'][0] - graph.node[v]['position'][0])**2 +  (graph.node[goal]['position'][1] - graph.node[v]['position'][1])**2  )


    def build_next_level(paths, frontier, explored, costIn, graph, goal):
        p = copy.deepcopy(paths)
        ex = copy.deepcopy(explored)
        mf = copy.deepcopy(frontier)
        cst = copy.deepcopy(costIn)

        node, prty = mf.pop()

        for child in graph.neighbors(node):
            child_cost = cst[node] + graph.edge[node][child]['weight']

            if child not in cst or child_cost < cst[child]:
                cst[child] = child_cost
                priority = child_cost + heuristic(graph, node, child, goal)
                mf.put(priority, child)
                p[child] = node

        ex.append(node)
        return p, mf, ex, cst

    if start == goal:
            return [],([],[])

    #if they are neighbors
    if graph.neighbors(start).__contains__(goal):
        return [start,goal], ([start,goal],[goal,start])

    middleNodes = []

    startFrontier = PriorityQueue()
    startFrontier.put(0,start) #priority, node
    goalFrontier = PriorityQueue()
    goalFrontier.put(0,goal)

    start_paths = { start:None } # node:parent
    goal_paths = { goal:None }

    sCost = {start:0}
    gCost = {goal:0}

    sExplored = []
    gExplored = []

    while startFrontier.has_element() and goalFrontier.has_element():
        # get shared nodes
        middleNodes = list(set(sExplored) & set(gExplored))
        if middleNodes: break

        # build next level/depth decide on which to expand
        sNode, sPriority = startFrontier.pop()
        startFrontier.put(sPriority, sNode)
        gNode, gPriority = goalFrontier.pop()
        goalFrontier.put(gPriority, gNode)

        if sCost[sNode] <= gCost[gNode]: #expand from start side
            start_paths, startFrontier, sExplored, sCost= build_next_level(start_paths, startFrontier, sExplored, sCost, graph, goal)
        else:
            goal_paths, goalFrontier, gExplored, gCost = build_next_level(goal_paths, goalFrontier, gExplored, gCost, graph, goal)
    else: # exited without break, empty path
        return [],([],[])

    # build the result

    middle = list(middleNodes)[0]
    #from middle to a
    res = []
    c = middle
    while c != start:
        c = start_paths[c]
        res.insert(0,c)

    res.append(middle)

    #from middle to b
    c = middle
    while c != goal:
        c = goal_paths[c]
        res.append(c)

    #look for shortcut
    #res is original path
    while startFrontier.has_element():
        node, prty = startFrontier.pop()
        if gExplored.__contains__(node):
            tempPath = [start]
            tempPath.extend(getPath(node, start, start_paths)[::-1])
            tempPath.extend(getPath(node, goal,goal_paths))
            tempPath = remove_dups(tempPath)
            tempCost = getCost(graph, tempPath)
            #Path ['h', 'u', 'b', 'f', 's'] is longer than an optimal path ['h', 'u', 'b', 'p', 'r', 's']
            oCost = getCost(graph, remove_dups(res))
            if tempCost < oCost:
                res = tempPath

    return res, (sExplored,gExplored)

Exercise 3: Tridirectional search
------
20 points

Implement tridirectional search in the naive way: starting from each goal node, perform a uniform-cost search and keep expanding until two of the three searches meet.

This will return the path computed and the set of all nodes explored.

In [ ]:
def tridirectional_search(graph, goals):
    """Run tridirectional uniform-cost search
    between the goals, and return the path 
    and the nodes explored."""
    if len(goals)!= len(set(goals)):
        return [],[]
    
    paths = []    
    frontier = copy.copy(goals)
    explored = []
    costs = {goals:0}
    while frontier.has_element():
        node = frontier.pop(0)
        
        if node in explored:
            node2 = explored[explored.index(node)]
            path = [node, node2, costs[node] + costs[node2]]
            if (path not in paths):
                paths.append(path)
        
        explored.append(node)
        
        if ((len(paths) == (len(goals))-1)):
            break
        
        for edge in graph.edges(node):
            child = edge[1]
            if child not in explored:
                costs[child] += distance(node, child)
                if child not in frontier:
                    frontier.append(child)
                else:
                    child2 = frontier[frontier.index(child)]
                    if (costs[child] < costs[child2]):
                        frontier[frontier.index(child)] = child
        frontier = sorted(frontier, key=lambda x: x.cost)
        
    if ((len(paths) < (len(goals)-1))):
        return [],[]
    return paths, explored

Exercise 4: Tridirectional search
------
15 points

This is the heart of the assignment. Implement tridirectional search in such a way as to consistently improve on the performance of your previous implementation. This means consistently exploring fewer nodes during your search in order to reduce runtime.

The specifics are up to you, but we have a few suggestions:
- Tridirectional A*
- choosing landmarks and precomputing reach values
- ATL (A\*, landmarks, and triangle-inequality)
- shortcuts (skipping nodes with low reach values)

This function will return the path computed and the set of all nodes explored.

In [ ]:
def tridirectional_search_advanced(graph, goals):
    """Run an improved tridirectional search between
    goals, and return the path and nodes explored."""
    if len(goals)!= len(set(goals)):
        return [],[]
    
    paths = []    
    frontier = copy.copy(goals)
    explored = []
    costs = {goals:0}
    while frontier.has_element():
        node = frontier.pop(0)
        
        if node in explored:
            node2 = explored[explored.index(node)]
            path = [node, node2, costs[node] + costs[node2]]
            if (path not in paths):
                paths.append(path)
        
        explored.append(node)
        
        #Check if min paths found
        if ((len(paths) == (len(goals))-1)):
            break
        
        for edge in graph.edges(node):
            child = edge[1]
            if child not in explored:
                costs[child] += distance(node, child)
                if child not in frontier:
                    frontier.append(child)
                else:
                    child2 = frontier[frontier.index(child)]
                    if (costs[child] < costs[child2]):
                        frontier[frontier.index(child)] = child
        frontier = sorted(frontier, key=lambda x: x.cost)
        
    if ((len(paths) < (len(goals)-1))):
        return [],[]
    return paths, explored

Race!
---
Here's your chance to show us your best stuff. This part is mandatory if you want to compete in the race for extra credit. Implement custom_search() using whatever strategy you'd like. Remember that 'goals' will be a list of the three nodes between which you should route.

In [ ]:
def custom_search(graph, goals):
    """Run your best tridirectional search between
    goals, and return the path and nodes explored."""
    raise NotImplementedError
    # return path, nodes_explored